In [1]:
import numpy as np 
import math

In [2]:
L,d_k,d_V = 4,8,8
q = np.random.randn(L,d_k)
k = np.random.randn(L,d_k)
v = np.random.randn(L,d_V)


In [3]:
print("Q\n",q)
print("W\n",k)
print("V\n",v)

Q
 [[ 1.47079544  0.94544991  0.11432364 -0.82264383 -1.07690323 -0.8377866
   0.89731589 -0.73518777]
 [ 0.30174324 -0.46393619  2.21484236 -2.54614492 -1.39637666  0.07252524
  -0.24162127 -0.5056483 ]
 [-0.65268342 -0.13847951  0.58517388 -0.54466552  0.24413499 -0.99231483
  -1.61469511  0.7233173 ]
 [ 1.29430289  0.30889733  0.64249662  2.01483537 -0.76045713  0.61302165
  -0.76062154 -0.71551973]]
W
 [[ 0.29847428  0.56445131  0.47308996 -0.55928965 -2.08418923  0.02623891
  -2.03140845  1.00126964]
 [ 1.99073209  1.25118768  0.16734232  1.08565506 -1.6965257   0.90926172
  -0.61492394  0.04818199]
 [ 1.27535146  1.07219187  0.73637664 -0.38762256  1.10281235 -1.21727865
  -0.08541894  0.09110149]
 [-0.05756785  0.99959559 -0.19946572  0.35050611 -1.37494223 -1.55371739
  -0.26602491  1.20038896]]
V
 [[-0.42432474 -0.39542769 -0.36987668 -0.0304305  -1.07674195  0.86780069
   1.38603063  1.1127637 ]
 [-0.96362591  0.06577554 -0.51002193 -2.16569437  0.71217977 -0.6653745
  -0.822

In [5]:
#Self Attention

np.matmul(q,k.T) #softmax(Q.K^T/d_k**0.5 +M)V


array([[ 1.15038784,  3.7149416 ,  2.98111849,  2.21039539],
       [ 5.19680278,  0.18576933,  0.85165031, -0.5507876 ],
       [ 3.77797167, -2.25466583,  1.34213523,  2.09543587],
       [ 2.16748217,  7.53881335,  0.08893905,  0.24888275]])

In [7]:
q.var(),k.var(), np.matmul(q,k.T).var()

(1.0715697304249114, 1.0688839314076906, 5.314866376710878)

In [8]:
scaled = np.matmul(q,k.T)/math.sqrt(d_k)
q.var(),k.var(),scaled.var()

(1.0715697304249114, 1.0688839314076906, 0.6643582970888596)

In [10]:
#Masking  -it is required in decoding part 

mask = np.tril(np.ones((L,L)))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [11]:
mask[mask == 0] = -np.infty
mask[mask == 1] = 0


In [12]:
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [13]:
scaled + mask

array([[ 0.40672352,        -inf,        -inf,        -inf],
       [ 1.83734724,  0.06567938,        -inf,        -inf],
       [ 1.33571469, -0.79714475,  0.47451646,        -inf],
       [ 0.76632067,  2.66537302,  0.0314447 ,  0.08799334]])

In [14]:
#softmax 

def softmax(x):
    return (np.exp(x).T/np.sum(np.exp(x),axis = -1)).T
attention = softmax(scaled+mask)

In [15]:
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.85466496, 0.14533504, 0.        , 0.        ],
       [0.64886471, 0.07688915, 0.27424614, 0.        ],
       [0.11538562, 0.77072532, 0.0553349 , 0.05855417]])

In [16]:
new_v = np.matmul(attention,v)
new_v

array([[-0.42432474, -0.39542769, -0.36987668, -0.0304305 , -1.07674195,
         0.86780069,  1.38603063,  1.1127637 ],
       [-0.5027041 , -0.3283987 , -0.39024469, -0.34075915, -0.81674895,
         0.64497662,  1.06508968,  0.81577136],
       [-0.19665329, -0.16354946, -0.75803482, -0.11896673, -0.68196965,
         0.12633495,  0.77628734,  0.36293678],
       [-0.87298758, -0.09119581, -0.60381935, -1.67275553,  0.34020954,
        -0.36129294, -0.45744014, -0.6130744 ]])

In [17]:
def scaled_dot_product_attention(q,k,v,mask=None):
    d_k = q.shape[-1]
    scaled = np.matmul(q,k.T)/math.sqrt(d_k)
    if mask is not None:
        scaled += mask
    attention = softmax(scaled)
    out = np.matmul(attention,v)
    return out,attention

In [18]:
values,attention = scaled_dot_product_attention(q,k,v,mask=mask)
print("values.shape:",values.shape)
print("attention.shape:",attention.shape)
print("Values",values)
print("attention",attention)

values.shape: (4, 8)
attention.shape: (4, 4)
Values [[-0.42432474 -0.39542769 -0.36987668 -0.0304305  -1.07674195  0.86780069
   1.38603063  1.1127637 ]
 [-0.5027041  -0.3283987  -0.39024469 -0.34075915 -0.81674895  0.64497662
   1.06508968  0.81577136]
 [-0.19665329 -0.16354946 -0.75803482 -0.11896673 -0.68196965  0.12633495
   0.77628734  0.36293678]
 [-0.87298758 -0.09119581 -0.60381935 -1.67275553  0.34020954 -0.36129294
  -0.45744014 -0.6130744 ]]
attention [[1.         0.         0.         0.        ]
 [0.85466496 0.14533504 0.         0.        ]
 [0.64886471 0.07688915 0.27424614 0.        ]
 [0.11538562 0.77072532 0.0553349  0.05855417]]
